### VDJ recombination statistics

In [1]:
import os
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from Bio import SeqIO
from antpack import SingleChainAnnotator, SequenceScoringTool

aligner = SingleChainAnnotator(chains=["H", "K", "L"], scheme = "imgt")

if "notebooks" in os.getcwd():
    current_dir = os.path.join(os.getcwd(), "..", "..")

score_tool = SequenceScoringTool()

os.chdir(current_dir)
    
if "humanization_test_data" not in os.getcwd():
    os.chdir(os.path.join(current_dir, "humanization_test_data"))

In [2]:
raw = pd.read_csv("immunesim_50k_default.csv")

In [4]:
raw.columns

Index(['sequence', 'sequence_aa', 'junction', 'junction_aa', 'v_call',
       'd_call', 'j_call', 'np1', 'np2', 'del_v', 'del_d_5', 'del_d_3',
       'del_j', 'v_sequence_alignment', 'd_sequence_alignment',
       'j_sequence_alignment', 'freqs', 'counts', 'shm_events',
       'name_repertoire'],
      dtype='object')